# Resnet-18 QAT TIMM

This notebook demonstrates on how to use a pretrained Resnet18 model available from the pytorch-image-model package, retraining the network for cats and dog dataset, and use quantization aware training to quantatize the model and later compiling it to the Open Vino IR format to be used for inference.

Import the necessary packages

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import nncf
from nncf.torch import create_compressed_model, register_default_init_args
import timm
from nncf import NNCFConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from openvino.runtime import Core
import cv2
from cat_dog import Custom_dataset
import numpy as np
from PIL import Image
import os
import time

# Dataset and Dataloader

Create a train_set and a test and load the data to pytorch Dataloader which will be used for training and validation

In [ ]:
train_dataset = Custom_dataset('train.csv','dataset/training_set',transform=transforms.ToTensor())
test_set = Custom_dataset('test.csv','dataset/test_set',transform=transforms.ToTensor())
trainloader = DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=8)
test_loader = DataLoader(train_dataset,batch_size=64,shuffle=False,num_workers=8)

# Training Resnet18 model on custom dataset
The training and testing part of the model have been implemented in the function given below for ease of use. The model will be trained on the cats and dogs dataset which consist of 2 classes and 8000 images for training and 2000 images for testing.

In [ ]:
def train(model, device, train_loader, optimizer,loss_fn,epoch):
    '''
    train the model
    '''
    model.train()
    counter = 0
    correct = 0
    print("Epoch "+str(epoch))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1,keepdim=False)
        correct += (pred == target).sum().item()
        counter += target.size(0)
    print("Accuracy after epoch %s is %s "%(epoch,100*correct/len(train_loader.dataset)))

def test(model, device, test_loader):
    '''
    test the model
    '''
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct, len(test_loader.dataset), acc))

# Getting the Pre-trained Resnet18 model from TIMM

To train the model on the custom dataset and later use it for QAT, we need to use a pretrained Resnet18 model. We obatin one from the Pytorch Image Model. We fine tune it to our datset by running it for 3 epochs.

In [ ]:
model = timm.create_model('resnet18',pretrained=True)
# Changing the final layer of the model for our custom class
model.fc = nn.Linear(512,2)
# Model is transfered to the GPU for faster training
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

Training the model for 3 epochs as it is observed that is fine tuned quickly

In [ ]:
epochs = 3
for epoch in range(epochs):
    train(model,device,trainloader,optimizer,loss,epoch)
    test(model,device,test_loader)

In [ ]:
#The pytorch weights have been saved inside the model folder
PATH = 'model/resnet_18_cat_dog.pth'
torch.save(model.state_dict(), PATH)

The FP32 pytorch model weights has been converted to ONNX format which will later be used to convert to OPENVINO IR format

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(model, dummy_input, 'model/fp32/resnet_18_cat_dog.onnx')
print(f"FP32 ONNX model was exported to model folder.")

In [ ]:
nncf_config_dict = {
    "input_info": {
      "sample_size": [1, 3, 224, 224]
    },
    "compression": {
        "algorithm": "quantization", 
    }
}

nncf_config = NNCFConfig.from_dict(nncf_config_dict)
# Provide data loaders for compression algorithm initialization, if necessary
nncf_config = register_default_init_args(nncf_config, trainloader)
# Apply the specified compression algorithms to the model
compression_ctrl, model = create_compressed_model(model, nncf_config)

In [ ]:
# We test the quantized model on the test set
acc1 = test(model, device, test_loader)

In [ ]:
# We finetune it for 1 epoch
epochs = 1
for epoch in range(epochs):
    train(model,device,trainloader,optimizer,loss,epoch)
    test(model,device,test_loader)

We then export our finetuned int8 model to ONNX format via the export_model function of compression_ctrl

In [ ]:
int_8_path = 'model/resnet_18_cat_dog_int_8.onnx'
compression_ctrl.export_model(int_8_path)

We then convert the FP32 and INT8 ONNX model to OpenVINO IR format which will later be used for benchmark usage.
We specify the input_model path via the --input_model and the output_dir where the model will be saved via the --output_dir. We also specify the input image dimension to the model via the --input_shape.

In [ ]:
!mo --input_model model/fp32/resnet_18_cat_dog.onnx --input_shape "[1,3, 224, 224]"  --data_type FP16 --output_dir model/fp32

In [ ]:
!mo --input_model model/int8/resnet_18_cat_dog_int_8.onnx --input_shape "[1,3, 224, 224]"  --data_type FP16 --output_dir model/int8

# Benchmarking FP32 vs INT8 model

We will now start benchmarking the FP32 model vs the INT8 model. We will do it via comparing the time taken for both the models to go through the entire test_set which consist of 1000 images. We also print frames-per-second achived from both the models. A benchmark function has been created which takes in the input image, passed it onto the model and gives us the output.

# Loading the model

In [ ]:
ie = Core()
model = ie.read_model(model="model/fp32/resnet_18_cat_dog.xml")

# To test the INT8 model, simply comment the model variable above and uncomment the variable below

# model = ie.read_model(model="model/int8/resnet_18_cat_dog_int_8.xml")
compiled_model = ie.compile_model(model=model, device_name="CPU")

output_layer = compiled_model.output(0)

# Preprocessing the input image

In [ ]:
def preprocess(image):
    img = cv2.cvtColor(cv2.imread(filename=image), code=cv2.COLOR_BGR2RGB)
    input_image = cv2.resize(src=img, dsize=(224, 224),interpolation = cv2.INTER_LINEAR)
    input_image = np.expand_dims(input_image.transpose(2, 0, 1), 0)
    input_image = input_image/255
    return input_image

# Benchmark function

In [ ]:
def benchmark(model,output_layer):
    img_dir = os.listdir('dataset/test_set')
    img_dir.sort()
    prediction_class = ('cat','dog')

    
    for i in os.listdir('dataset/test_set'):
        filename = 'dataset/test_set/'+ i
        img = preprocess(filename)
        output = model([img])[output_layer]
        result_index = np.argmax(output)
        print('filename = %s and prediction = %s '%(i,prediction_class[result_index]))

# Calculate the time taken and FPS obatained

In [ ]:
img_dir  = os.listdir('dataset/training_set')
start = time.time()
test_go = benchmark(compiled_model,output_layer)
end = time.time()
total_time = end-start
print('time taken = %s and FPS = %s '%(total_time,len(img_dir)/total_time))